In [1]:
!pip3 install nltk
!pip3 install Sastrawi
!pip3 install nlp_id

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 18.3 MB/s eta 0:00:00
  Created wheel for nlp_id: filename=nlp_id-0.1.15.0-py3-none-any.whl size=58153894 sha256=49fba302b6dda65c70b0bf445691f5a8b8ac385a2d7aa8ae83f01be38e01fce7
  Stored in directory: /root/.cache/pip/wheels/85/34/2a/deaf7b7896a1eed336b874e1a7732588d40bb7bdc0a824e73e
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=a3dec65f93e5819ed25d96851edb985939c9a84938e94410bb29924c7fbe57ce
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built nlp_id wget
  Attempting uninstall: pytest
    Found existing installation: pytest 7.2.2
    Uninstalling pytest-7.2.2:
      Successfully unins

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nlp_id.lemmatizer import Lemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.neural_network import MLPClassifier
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
alay_dict=pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/DATA/new_kamusalay.csv",encoding='latin1',header=None)
alay_dict=alay_dict.rename(columns={0:'Original',1:'Baku'})

kasar_dict=pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/DATA/abusive.csv",encoding='latin1')
kasar_dict['Kata_Sensor']="a1b2c3d4e5f6" #Inisiasi kata ganti untuk kata-kata yang kasar dengan kata "disensor"

print(kasar_dict)

alay_dict_map = dict(zip(alay_dict['Original'], alay_dict['Baku']))
kasar_dict_map = dict(zip(kasar_dict['ABUSIVE'],kasar_dict['Kata_Sensor']))

def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])


def sensor_kata_kasar(text):
    return ' '.join([kasar_dict_map[word] if word in kasar_dict_map else word for word in text.split(' ')])


def preprocess(TextYangInginDiPreProcess):
    #Tahap Pertama Adalah Membuat semua huruf menjadi huruf kecil atau lower
    text = TextYangInginDiPreProcess.lower()

    #Tahap Kedua adalah menghilangkan non alpha numeric character pada text
    text = re.sub('[^0-9a-zA-Z]+',' ',text)

    #Tahap Ketiga adalah menghilangkan char tidak penting
    text=re.sub('\n',' ',text) #Menghilangkan new line pada data
    text=re.sub('rt',' ',text) #Menghilangkan kata-kata retweet
    text=re.sub('user',' ',text) #Menghilangkan kata-kata user
    text=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) #Menghilangkan  URL
    text=re.sub(' +',' ',text) #Menghilangkan ekstra spasi

    #Tahap keempat adalah membuat map terhadap kata-kata "alay" dan mengubah nya menjadi kata yang baku
    text=normalize_alay(text)

    #Tahap kelima adalah mensensor kata kasar dengan kata "disensor"
    text=sensor_kata_kasar(text)
    return text

def menghilangkan_kata_umum_dengan_stopword(text):
    # Mengambil daftar stopwords bahasa Indonesia dari NLTK
    stop_words = set(stopwords.words('indonesian'))

    # Memisahkan teks menjadi kata-kata
    words = text.split()

    # Menghapus stopwords dari teks
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Menggabungkan kata-kata yang tersisa menjadi teks tanpa stopwords
    filtered_text = ' '.join(filtered_words)

    return filtered_text

def mengurangi_variasi_kata_dengan_steeming(teks):
    Stemming = StemmerFactory()
    Fact = Stemming.create_stemmer()
    teks = Fact.stem(teks)
    return teks

def preprocess_semua_kata(teks):
    teks = preprocess(teks)
    teks = menghilangkan_kata_umum_dengan_stopword(teks)
    teks = mengurangi_variasi_kata_dengan_steeming(teks)
    return teks

         ABUSIVE   Kata_Sensor
0           alay  a1b2c3d4e5f6
1          ampas  a1b2c3d4e5f6
2           buta  a1b2c3d4e5f6
3        keparat  a1b2c3d4e5f6
4         anjing  a1b2c3d4e5f6
..           ...           ...
120        rezim  a1b2c3d4e5f6
121        sange  a1b2c3d4e5f6
122       serbet  a1b2c3d4e5f6
123        sipit  a1b2c3d4e5f6
124  transgender  a1b2c3d4e5f6

[125 rows x 2 columns]


In [4]:
data_train = pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/Data_Yang_Telah_Di_Preprocess.csv")
data_train

,Unnamed: 0,tweet,sentiment
0,0,warung milik usaha pabrik puluh kenal putih ba...,positive
1,1,lokasi strategis jalan sumatra bandung nya nya...,positive
2,2,betapa bahagia nya unboxing paket barang nya b...,positive
3,3,makan agam harga makan food stall 10 kasir sua...,positive
4,4,unik bagus foto makan enak pegawai ramah bersi...,positive
...,...,...,...
3439,3439,a1b2c3d4e5f6 anjing orang orang sarawak a1b2c3...,negative
3440,3440,iri fungsi sekolah ubah sepe i,negative
3441,3441,halo j triliun butuh tunggu 24 jam jaka a tar ...,negative
3442,3442,agama buddha sesat serah debat orang a1b2c3d4e...,negative


In [5]:
data_train.isna().sum()

Unnamed: 0    0
tweet         1
sentiment     0
dtype: int64

In [6]:
data_train = data_train.dropna()

In [7]:
data_train.isna().sum()

Unnamed: 0    0
tweet         0
sentiment     0
dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_train['tweet'], data_train['sentiment'], test_size=0.2, random_state=42)

In [9]:
X_train

2401    maklum belah pola pikir nya sempit otak nya tempe
3367    warga kampung pulo muara zalim cina rata daera...
2615    ramai masuk daftar tunggu 10 menit telepon kos...
999     senang kumpul teman teman pasang makan minum s...
2382    sih a1b2c3d4e5f6 pimpin diketawain moderator n...
                              ...                        
1095    bakmi favorit makan pasang menu favorit bakmi ...
1130    bakso nya enak pilih sesuai selera nya strateg...
1294    asyik tumbang politikus gerindra 2019 ganti pr...
860     makan pesan via delivery order anak suka manta...
3175                         kecewa layan gojek indonesia
Name: tweet, Length: 2754, dtype: object

In [10]:
X_test

790                          baca 543 halaman roman bosan
51      bihun goreng nya enak minyak makan kandung a1b...
1091    kilas sop durian lodaya makan durian waralaba ...
554     suasana nya enak banget tenang mes kue serabi ...
1815      sopir angkut kota tunggu anies tanah abang temu
                              ...                        
2354    enak selera ubah mempe ahankan tradisi nya lay...
73      enak banget wajib coba cinta ayam bakar harga ...
48      lumayan sambal nya enak saji 3 model pilih sih...
1725    wabah difteri anies baswedan ajak warga responsif
1503                  halo mohon maaf ketidaknyamanan nya
Name: tweet, Length: 689, dtype: object

In [11]:
import pickle
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

with open("/content/drive/MyDrive/Program Platinum Challange/"+"vectorizer.pkl", "wb") as file:
    pickle.dump(vectorizer, file)

In [12]:
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)

In [13]:
model.fit(X_train, y_train)

MLPClassifier(max_iter=1000, random_state=42)

In [14]:
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7692307692307693


In [15]:
text = "Dasar anjing anak itu"
preprocessed_text = preprocess_semua_kata(text)
text_vector = vectorizer.transform([preprocessed_text])
sentiment_pred = model.predict(text_vector)

print("Teks:", text)
print("Teks yang di preprocess: ",preprocessed_text)
print("Sentimen Prediksi:", sentiment_pred)

Teks: Dasar anjing anak itu
Teks yang di preprocess:  dasar a1b2c3d4e5f6 anak
Sentimen Prediksi: ['negative']


In [16]:
from sklearn.metrics import classification_report
testing = model.predict(X_test)

print(classification_report(y_test,testing))

              precision    recall  f1-score   support

    negative       0.70      0.72      0.71       214
     neutral       0.77      0.78      0.78       215
    positive       0.83      0.80      0.81       260

    accuracy                           0.77       689
   macro avg       0.77      0.77      0.77       689
weighted avg       0.77      0.77      0.77       689



In [17]:
import pickle

with open("/content/drive/MyDrive/Program Platinum Challange/"+"model_MLPClassifier.pkl", "wb") as file:
    pickle.dump(model, file)

In [18]:
import pickle

# Muat model dari file pkl
with open("/content/drive/MyDrive/Program Platinum Challange/model_MLPClassifier.pkl", "rb") as file:
    model1 = pickle.load(file)

# Gunakan model untuk melakukan prediksi
text1 = "Dasar anjing anak itu"
preprocessed_text1 = preprocess_semua_kata(text1)
text_vector1 = vectorizer.transform([preprocessed_text1])
sentiment_pred1 = model1.predict(text_vector1)

print("Teks baru :", text)
print("Sentimen Prediksi:", sentiment_pred1)


Teks baru : Dasar anjing anak itu
Sentimen Prediksi: ['negative']


In [19]:
def lakukan_prediksi(teks_masukan,model=model1):
  text_vector2 = vectorizer.transform([teks_masukan])
  sentiment_pred1 = model1.predict(text_vector2)
  return sentiment_pred1

In [20]:
data_uji_coba = pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/Data_Testing_Yang_Telah_Di_Preprocess.csv")
data_uji_coba

,Unnamed: 0,Tweet
0,0,cowok usaha lacak perhati gue lantas remeh per...
1,1,telat tau a1b2c3d4e5f6 a1b2c3d4e5f6 gue gaul c...
2,2,41 kadang pikir percaya tuhan jatuh kali kali ...
3,3,ku tau mata a1b2c3d4e5f6
4,4,kaum a1b2c3d4e5f6 a1b2c3d4e5f6 dongok a1b2c3d4...
...,...,...
13164,13164,bicara ndasmu a1b2c3d4e5f6 sekata a1b2c3d4e5f6
13165,13165,kasur enak a1b2c3d4e5f6
13166,13166,hati hati a1b2c3d4e5f6 bosan duh xf0 x9f x98 xaa
13167,13167,bom real mudah deteksi bom kubur dahsyat ledak...


In [21]:
data_uji_coba = data_uji_coba.dropna()

In [22]:
prediksi_sentimen = data_uji_coba['Tweet'].apply(lakukan_prediksi)

In [23]:
kk = prediksi_sentimen[0]

In [24]:
kk[0]

'neutral'

In [25]:
len(prediksi_sentimen)

13121

In [26]:
jumlah_negative = 0
jumlah_positive = 0
jumlah_neutral  = 0

for j in prediksi_sentimen:
  if j[0]=='negative':
    jumlah_negative +=1
  if j[0]=='neutral':
    jumlah_neutral +=1
  if j[0]=='positive':
    jumlah_positive +=1

In [27]:
print("Jumlah Negative : " + str(jumlah_negative))
print("Jumlah Positive : "+ str(jumlah_positive))
print("Jumlah Neutral : "+str(jumlah_neutral))

Jumlah Negative : 8513
Jumlah Positive : 1604
Jumlah Neutral : 3004
